In [11]:
# requires gym, pygame
import gym, os
from genepro.node_impl import *
from genepro.evo import Evolution

# Set up the environment in which experiments will take place
env_name = "CartPole-v1"
env = gym.make(env_name)

# Let's set up the possible nodes for GP
num_features = env.observation_space.shape[0] + 1 # observations from environment + action at time t-1
leaf_nodes = [Feature(i) for i in range(num_features)]
leaf_nodes = leaf_nodes + [Constant()] # allow to use random constants
internal_nodes = [Plus(),Minus(),Times(),Div(),Log()]

# create a fitness function for that environment
def fitness_function(tree, num_episodes=10, episode_duration=500, render=False, ignore_done=False):
  rewards = list()
  durations = list()
  for _ in range(num_episodes):
    # get initial state
    observation = env.reset()
    # we do not have an action at time -1, let's set it randomly
    action = env.action_space.sample()
    for t in range(episode_duration):
      if render:
        env.render()
      # build up the input sample for GP
      input_sample = np.concatenate((observation, [action])).reshape((1,-1))
      # get output (squeezing because it is encapsulated in an array)
      output = tree.get_output(input_sample).astype(float).squeeze()
      action = 0 if output < .5 else 1
      observation, reward, done, _ = env.step(action)
      rewards.append(reward)
      if done and not ignore_done:
        break
    # keep track of how long this lasted
    durations.append(t)

  # compute and return fitness
  fitness = np.sum(rewards)
  return fitness


In [12]:
# set up and run the evolution
evo = Evolution(
  fitness_function, internal_nodes, leaf_nodes, 
  pop_size=128,
  max_gens=20,
  max_tree_size=31,
  n_jobs=4,
  verbose=True)
evo.evolve()

gen: 1,	best of gen fitness: 1126.000,	best of gen size: 16
gen: 2,	best of gen fitness: 873.000,	best of gen size: 16
gen: 3,	best of gen fitness: 1090.000,	best of gen size: 19
gen: 4,	best of gen fitness: 4673.000,	best of gen size: 21
gen: 5,	best of gen fitness: 3876.000,	best of gen size: 21
gen: 6,	best of gen fitness: 4764.000,	best of gen size: 21
gen: 7,	best of gen fitness: 5000.000,	best of gen size: 21
gen: 8,	best of gen fitness: 5000.000,	best of gen size: 21
gen: 9,	best of gen fitness: 5000.000,	best of gen size: 30
gen: 10,	best of gen fitness: 5000.000,	best of gen size: 21
gen: 11,	best of gen fitness: 5000.000,	best of gen size: 21
gen: 12,	best of gen fitness: 5000.000,	best of gen size: 23
gen: 13,	best of gen fitness: 5000.000,	best of gen size: 28
gen: 14,	best of gen fitness: 5000.000,	best of gen size: 27
gen: 15,	best of gen fitness: 5000.000,	best of gen size: 27
gen: 16,	best of gen fitness: 5000.000,	best of gen size: 28
gen: 17,	best of gen fitness: 5000

In [13]:
# fetch elite
best = evo.best_of_gens[-1]
print(best.get_readable_repr())

((((x0/x0)+x4)+(x3+(x2*x3)))*(((((x2+x3)*log(abs(x2)))*log(abs(x3)))*log(abs(x2)))*log(abs(x3))))


In [14]:
# see simplified representation (if possible, i.e., symbols are compatible)
from sympy import simplify
simpl_repr = simplify(best.get_readable_repr())
print(simpl_repr)

(x2 + x3)*(x2*x3 + x3 + x4 + 1)*log(abs(x2))**2*log(abs(x3))**2


In [16]:
# you can try the line below, but does not work on some machines
#fitness_function(elite, num_episodes=3, episode_duration=1000, render=True, ignore_done=True)

# alternatively, what follows uses an external evaluation script
os.system("python run_gym_tree.py '"+str(best.get_subtree())+"'")

0